In [1]:
import pandas as pd
import numpy as np
import requests
import os
import psycopg2
import configparser
import config

In [2]:
res2 = requests.get("http://isin.twse.com.tw/isin/C_public.jsp?strMode=2")
res4 = requests.get("http://isin.twse.com.tw/isin/C_public.jsp?strMode=4")

In [3]:
df2 = pd.read_html(res2.text)[0]
df4 = pd.read_html(res4.text)[0]

df2.columns = df2.iloc[0]
df4.columns = df4.iloc[0]

df2 = df2.iloc[1:]
df4 = df4.iloc[1:]

df2 = df2[df2['產業別'].notna()]
df2 = df2[df2['CFICode']=='ESVUFR']

df4 = df4[df4['產業別'].notna()]
df4 = df4[df4['CFICode']=='ESVUFR']

df2_arr = np.array(df2['有價證券代號及名稱'])
df4_arr = np.array(df4['有價證券代號及名稱'])

df2_arr = np.array([df2_arr])
df4_arr = np.array([df4_arr])

In [56]:
TSE_Amount = df2_arr.size
TSE = np.empty(TSE_Amount*2,dtype = object).reshape(TSE_Amount,2)

OTC_Amount = df4_arr.size
OTC = np.empty(OTC_Amount*2,dtype = object).reshape(OTC_Amount,2)

In [57]:
# for i in range(TSE_Amount):
#     TSE[i][0]=df2_arr[0][i][:4]
#     TSE[i][1]=df2_arr[0][i][5:]
    
# for i in range(OTC_Amount):
#     OTC[i][0]=df4_arr[0][i][:4]
#     OTC[i][1]=df4_arr[0][i][5:]

In [58]:
filepath = os.getcwd() + r'\Stock_SQL'

INITXT=filepath + r"\config.ini"
config = configparser.ConfigParser()
config.read(INITXT)
host    = config.get("POSTGRES","host")
user    = config.get("POSTGRES","user")
passwd  = config.get("POSTGRES","passwd")
db      = config.get("POSTGRES","db")

# list(config['POSTGRES'])
# list(config)
# host = config['POSTGRES']['host']

In [69]:
try:
    connection = psycopg2.connect(user="admin",
                                  password="123456",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="mockdb")

    cursor = connection.cursor()
    
    for i in range(TSE_Amount):
        stock_id=df2_arr[0][i][:4]
        stock_name=df2_arr[0][i][5:]
        create_table_query = '''
        insert into {stock_list}
        select {stock_id},'{stock_name}'
        where not exists(select stock_id from {TSE_OTC} where stock_id = {stock_id} );
        '''.format(stock_list = 'tse',stock_id=stock_id,stock_name=stock_name,TSE_OTC = 'TSE')
        cursor.execute(create_table_query)
        connection.commit()
    
    for i in range(OTC_Amount):
        stock_id=df4_arr[0][i][:4]
        stock_name=df4_arr[0][i][5:]
        create_table_query = '''
        insert into {stock_list}
        select {stock_id},'{stock_name}'
        where not exists(select stock_id from {TSE_OTC} where stock_id = {stock_id} );
        '''.format(stock_list = 'otc',stock_id=stock_id,stock_name=stock_name,TSE_OTC = 'OTC')
        cursor.execute(create_table_query)
        connection.commit()
    

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    if connection:
        cursor = connection.cursor()
        cursor.close()
        connection.close()